In [ ]:
import os
import csv
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.windows import Window
from shapely.geometry import box
import geopandas as gpd
import cv2
from tqdm import tqdm
import shutil

# --- Config ---
map_folder = "Bear_Lane"
base_dir = "C:/QGIS"
chip_size = 1024
stride = 922

# Original → YOLO class IDs
class_remap = {
    1: 0,  # Road
    2: 1,  # PVeg
    3: 2   # Water
}

map_base = os.path.join(base_dir, map_folder)
tif_path = os.path.join(map_base, f"{map_folder}.tiff")
shp_path = os.path.join(map_base, f"{map_folder}.shp")

out_img_dir = os.path.join(map_base, "tiled", "images")
out_mask_dir = os.path.join(map_base, "tiled", "masks")
meta_csv = os.path.join(map_base, "tiled", "tile_metadata.csv")
shape_txt = os.path.join(map_base, "tiled", "raster_shape.txt")

# --- Clean old data ---
for path in [out_img_dir, out_mask_dir]:
    if os.path.exists(path):
        shutil.rmtree(path)
os.makedirs(out_img_dir, exist_ok=True)
os.makedirs(out_mask_dir, exist_ok=True)

if os.path.exists(meta_csv):
    os.remove(meta_csv)
if os.path.exists(shape_txt):
    os.remove(shape_txt)

# --- Open raster and store shape ---
with rasterio.open(tif_path) as raster:
    raster_height, raster_width = raster.height, raster.width
    raster_crs = raster.crs
    with open(shape_txt, "w") as f:
        f.write(f"{raster_height},{raster_width}")

# --- Load and reproject shapefile ---
labels = gpd.read_file(shp_path).to_crs(raster_crs)
sindex = labels.sindex

# --- Begin processing ---
results = []
skipped = 0

with rasterio.open(tif_path) as raster:
    for idx, (y, x) in enumerate(tqdm(
        [(y, x) for y in range(0, raster_height - chip_size + 1, stride)
                 for x in range(0, raster_width - chip_size + 1, stride)],
        desc="🌍 Chipping tiles"
    )):
        window = Window(x, y, chip_size, chip_size)
        transform = raster.window_transform(window)
        bounds = box(*rasterio.windows.bounds(window, raster.transform))

        # Read image chip
        img = raster.read([1, 2, 3], window=window)
        img = np.transpose(img, (1, 2, 0))
        img_name = f"chip_{idx}.png"

        # --- Skip writing black tiles, but keep metadata ---
        if np.mean(img) < 5 and np.std(img) < 2:
            skipped += 1
            results.append((img_name, x, y))
            continue

        # Save image
        cv2.imwrite(os.path.join(out_img_dir, img_name), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

        # Rasterize mask
        possible_matches = labels.iloc[list(sindex.intersection(bounds.bounds))]
        intersecting = possible_matches[possible_matches.intersects(bounds)]
        if not intersecting.empty:
            shapes = [
                (geom, class_remap.get(cid, 255))
                for geom, cid in zip(intersecting.geometry, intersecting["class_id"])
            ]
            mask = rasterize(
                shapes,
                out_shape=(chip_size, chip_size),
                transform=transform,
                fill=255,
                dtype=np.uint8
            )
        else:
            mask = np.full((chip_size, chip_size), 255, dtype=np.uint8)

        cv2.imwrite(os.path.join(out_mask_dir, img_name), mask)
        results.append((img_name, x, y))

# --- Save metadata ---
with open(meta_csv, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "x", "y"])
    writer.writerows(results)

print(f"✅ {len(results)} tiles processed.")
print(f"🧹 {skipped} black/void tiles skipped (not saved, but tracked for stitching).")

🌍 Chipping tiles:  69%|████████████████████████████████████████▉                  | 1021/1470 [04:09<02:44,  2.74it/s]

In [ ]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- Configuration ---
base_dir = "C:/QGIS"
target_size = 640
num_threads = 8

all_maps = [
    "Bear_Creek_20250112",
    "Bear_Lane",
    "Flight_2",
    "Flight_2_25pct",
    "SFLBC",
    "Sugar_Refugia_20241112",
    "Wildcat_Creek",
    "Project_2024_09_20"
]

def mask_to_polygons(mask):
    contours = {}
    for cls_id in np.unique(mask):
        if cls_id == 255:
            continue
        binary = (mask == cls_id).astype(np.uint8)
        cnts, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if cnts:
            contours[cls_id] = cnts
    return contours

def process_file(fname, img_input_dir, mask_input_dir, img_out, lbl_out):
    img_path = os.path.join(img_input_dir, fname)
    mask_path = os.path.join(mask_input_dir, fname)

    try:
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if img is None or mask is None:
            return "error"

        img_resized = cv2.resize(img, (target_size, target_size), interpolation=cv2.INTER_AREA)
        mask_resized = cv2.resize(mask, (target_size, target_size), interpolation=cv2.INTER_NEAREST)

        label_path = os.path.join(lbl_out, fname.replace(".png", ".txt"))
        contours = mask_to_polygons(mask_resized)

        # Save image
        cv2.imwrite(os.path.join(img_out, fname), img_resized)

        # Write label file
        with open(label_path, "w") as f:
            if not contours:
                return "background"
            for cls_id, cnts in contours.items():
                for cnt in cnts:
                    if len(cnt) < 3:
                        continue
                    pts = cnt.reshape(-1, 2).astype(np.float32) / target_size
                    coords = " ".join(f"{x:.6f} {y:.6f}" for x, y in pts)
                    f.write(f"{cls_id} {coords}\n")

        return "labeled"
    except Exception as e:
        print(f"⚠️ Error processing {fname}: {e}")
        return "error"

# --- Process each map ---
for map_folder in all_maps:
    print(f"\n🧩 Converting: {map_folder}")

    base_map_dir = os.path.join(base_dir, map_folder)
    img_input_dir = os.path.join(base_map_dir, "tiled", "images")
    mask_input_dir = os.path.join(base_map_dir, "tiled", "masks")
    out_base = os.path.join(base_map_dir, "yolo_dataset_640")

    out_dirs = {
        "train_img": os.path.join(out_base, "images", "train"),
        "val_img": os.path.join(out_base, "images", "val"),
        "train_lbl": os.path.join(out_base, "labels", "train"),
        "val_lbl": os.path.join(out_base, "labels", "val"),
    }
    for d in out_dirs.values():
        os.makedirs(d, exist_ok=True)

    chip_files = [f for f in os.listdir(img_input_dir) if f.endswith(".png")]
    random.shuffle(chip_files)
    split_idx = int(len(chip_files) * 0.7)
    train_files = chip_files[:split_idx]
    val_files = chip_files[split_idx:]

    for mode, files in [("train", train_files), ("val", val_files)]:
        img_out = out_dirs[f"{mode}_img"]
        lbl_out = out_dirs[f"{mode}_lbl"]
        counts = {"labeled": 0, "background": 0, "error": 0}

        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            futures = [executor.submit(process_file, f, img_input_dir, mask_input_dir, img_out, lbl_out) for f in files]
            for f in tqdm(as_completed(futures), total=len(futures), desc=f"{map_folder} [{mode}]"):
                result = f.result()
                counts[result] += 1

        print(f"✅ {map_folder} [{mode}] summary:")
        print(f"   - {counts['labeled']} labeled tiles")
        print(f"   - {counts['background']} background-only tiles")
        print(f"   - {counts['error']} errors")


🧩 Converting: Bear_Creek_20250112


Bear_Creek_20250112 [train]:  13%|██████▌                                           | 669/5107 [00:12<01:33, 47.72it/s]

In [6]:
import os
import shutil
import random
from tqdm import tqdm

# --- Configuration ---
base_dir = "C:/QGIS"
dataset_dir = os.path.join(base_dir, "dataset")
img_train_dir = os.path.join(dataset_dir, "images", "train")
img_val_dir = os.path.join(dataset_dir, "images", "val")
lbl_train_dir = os.path.join(dataset_dir, "labels", "train")
lbl_val_dir = os.path.join(dataset_dir, "labels", "val")
manifest_dir = os.path.join(dataset_dir, "manifests")
train_split = 0.8

holdout_set = "Flight_2_25pct"
all_sets = [
    "Bear_Creek_20250112",
    "Bear_Lane",
    "Flight_2",
    "Flight_2_25pct",
    "SFLBC",
    "Sugar_Refugia_20241112",
    "Wildcat_Creek",
    "Project_2024_09_20"
]  # Excludes holdout

# --- Create necessary directories ---
for d in [img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir, manifest_dir]:
    os.makedirs(d, exist_ok=True)

# --- Clean old contents ---
for folder in [img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir]:
    for f in os.listdir(folder):
        file_path = os.path.join(folder, f)
        if os.path.isfile(file_path):
            os.remove(file_path)

for manifest_file in ["train_files.txt", "val_files.txt"]:
    path = os.path.join(manifest_dir, manifest_file)
    if os.path.exists(path):
        os.remove(path)

train_list, val_list = [], []

# --- Merge YOLO image/label files from each map set ---
for set_name in tqdm(all_sets, desc="Merging sets"):
    base_map_dir = os.path.join(base_dir, set_name)
    img_base = os.path.join(base_map_dir, "yolo_dataset_640", "images")
    lbl_base = os.path.join(base_map_dir, "yolo_dataset_640", "labels")

    all_images = []
    for split in ["train", "val"]:
        img_dir = os.path.join(img_base, split)
        lbl_dir = os.path.join(lbl_base, split)
        if os.path.exists(img_dir):
            for fname in os.listdir(img_dir):
                if fname.endswith(".png"):
                    all_images.append((
                        os.path.join(img_dir, fname),
                        os.path.join(lbl_dir, fname.replace(".png", ".txt"))
                    ))

    random.shuffle(all_images)
    split_idx = int(len(all_images) * train_split)
    split_files = [("train", all_images[:split_idx]), ("val", all_images[split_idx:])]

    for mode, files in split_files:
        img_out_dir = img_train_dir if mode == "train" else img_val_dir
        lbl_out_dir = lbl_train_dir if mode == "train" else lbl_val_dir
        manifest = train_list if mode == "train" else val_list

        for img_path, lbl_path in files:
            new_img_name = f"{set_name}_{os.path.basename(img_path)}"
            new_lbl_name = new_img_name.replace(".png", ".txt")

            shutil.copy(img_path, os.path.join(img_out_dir, new_img_name))
            if os.path.exists(lbl_path):
                shutil.copy(lbl_path, os.path.join(lbl_out_dir, new_lbl_name))
            else:
                open(os.path.join(lbl_out_dir, new_lbl_name), "w").close()

            manifest.append(f"images/{mode}/{new_img_name}")

# --- Save manifest files ---
os.makedirs(manifest_dir, exist_ok=True)
with open(os.path.join(manifest_dir, "train_files.txt"), "w") as f:
    f.write("\n".join(sorted(train_list)) + "\n")
with open(os.path.join(manifest_dir, "val_files.txt"), "w") as f:
    f.write("\n".join(sorted(val_list)) + "\n")

print(f"✅ Merged {len(train_list)} train / {len(val_list)} val tiles into: {dataset_dir}")
print(f"ℹ️ Held-out map: {holdout_set}")

Merging sets: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:40<00:00, 40.31s/it]

✅ Merged 816 train / 204 val tiles into: C:/QGIS\dataset
ℹ️ Held-out map: Flight_2_25pct


In [ ]:
# --- Training done in Train.py 
import os
from ultralytics import YOLO
import torch

def main():
    torch.cuda.empty_cache()

    dataset_dir = "C:/QGIS/dataset"
    model_type = "yolov8s-seg.pt"
    img_size = 1024
    save_dir = "C:/QGIS/runs/segment/train"
    class_names = ["Water", "Road", "PVeg"]

    data_yaml = os.path.join(dataset_dir, "data.yaml")
    with open(data_yaml, "w") as f:
        f.write(f"path: {dataset_dir}\n")
        f.write("train: images/train\n")
        f.write("val: images/val\n")
        f.write(f"nc: {len(class_names)}\n")
        f.write(f"names: {class_names}\n")

    print(f"✅ data.yaml written to: {data_yaml}")

    model = YOLO(model_type)
    model.train(
        data=data_yaml,
        imgsz=img_size,
        epochs=50,
        batch=16,
        workers=2,
        amp=True,
        patience=5,
        device="cuda",
        save=True,
        save_period=-1,
        project=save_dir,
        name="main_model",
        verbose=True,
        plots=False,
        cache=True
    )

if __name__ == "__main__":
    main()


In [ ]:
# Model Report

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

# --- Paths ---
results_path = "C:/QGIS/runs/segment/train/main_model/results.csv"
report_path = "C:/QGIS/diagnostics/model_report.pdf"
os.makedirs(os.path.dirname(report_path), exist_ok=True)

# --- Load results and clean columns ---
df = pd.read_csv(results_path)
df.columns = df.columns.str.strip()
11
# --- Key epoch metrics ---
final_epoch = df.iloc[-1]
best_map_epoch = df["metrics/mAP50(B)"].idxmax()
best_row = df.iloc[best_map_epoch]

# --- Basic training config (update if needed) ---
training_config = {
    "Model": "YOLOv8n-seg",
    "Input Size": "640x640",
    "Epochs": len(df),
    "Batch Size": 8,
    "Optimizer": "SGD (default)",
    "Confidence Threshold": 0.1,
    "Tile Size": "1024x1024",
    "Mask Source": "Polygon label → Raster mask via cv2.fillPoly"
}

# --- Metrics to plot ---
metrics = {
    "metrics/mAP50(B)": "mAP@0.5",
    "metrics/mAP50-95(B)": "mAP@0.5–0.95",
    "metrics/precision(B)": "Precision",
    "metrics/recall(B)": "Recall"
}
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]

# --- Create PDF report ---
with PdfPages(report_path) as pdf:
    # Title + Summary page
    plt.figure(figsize=(11, 8.5))
    plt.text(0.5, 0.78, "Model Performance Report", ha="center", fontsize=24)
    plt.text(0.5, 0.70, "Project: River and Road Segmentation", ha="center", fontsize=14)
    plt.text(0.5, 0.63, f"Best mAP@0.5: {best_row['metrics/mAP50(B)']:.3f} at epoch {best_map_epoch}", ha="center", fontsize=12)
    plt.text(0.5, 0.57, f"Final Epoch Precision: {final_epoch['metrics/precision(B)']:.3f}", ha="center", fontsize=12)
    plt.text(0.5, 0.51, f"Final Epoch Recall: {final_epoch['metrics/recall(B)']:.3f}", ha="center", fontsize=12)
    plt.text(0.5, 0.45, "Report generated from YOLOv8 results.csv", ha="center", fontsize=10)
    plt.axis("off")
    pdf.savefig()
    plt.close()

    # Training configuration page
    fig, ax = plt.subplots(figsize=(11, 4))
    ax.axis("off")
    table_data = list(training_config.items())
    table = ax.table(cellText=table_data, colLabels=["Parameter", "Value"], loc="center")
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1.2, 1.4)
    plt.title("Training Configuration", fontsize=14)
    pdf.savefig()
    plt.close()

    # Metric plots
    for i, (key, label) in enumerate(metrics.items()):
        if key in df.columns:
            plt.figure(figsize=(10, 4))
            plt.plot(df[key], marker='o', linewidth=2, color=colors[i])
            plt.title(f"{label} Over Epochs")
            plt.xlabel("Epoch")
            plt.ylabel(label)
            plt.grid(True)
            plt.tight_layout()
            pdf.savefig()
            plt.close()

    # Performance summary table
    summary_data = {
        "Metric": ["mAP@0.5", "mAP@0.5–0.95", "Precision", "Recall"],
        "Final Epoch": [
            f"{final_epoch.get('metrics/mAP50(B)', 0):.3f}",
            f"{final_epoch.get('metrics/mAP50-95(B)', 0):.3f}",
            f"{final_epoch.get('metrics/precision(B)', 0):.3f}",
            f"{final_epoch.get('metrics/recall(B)', 0):.3f}"
        ],
        f"Best Epoch ({best_map_epoch})": [
            f"{best_row.get('metrics/mAP50(B)', 0):.3f}",
            f"{best_row.get('metrics/mAP50-95(B)', 0):.3f}",
            f"{best_row.get('metrics/precision(B)', 0):.3f}",
            f"{best_row.get('metrics/recall(B)', 0):.3f}"
        ]
    }
    summary_df = pd.DataFrame(summary_data)
    fig, ax = plt.subplots(figsize=(10, 2.5))
    ax.axis("off")
    table = ax.table(cellText=summary_df.values, colLabels=summary_df.columns, loc="center")
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 1.8)
    plt.title("Summary of Model Performance", fontsize=14)
    pdf.savefig()
    plt.close()

print(f"✅ PDF report saved to: {report_path}")

✅ PDF report saved to: C:/QGIS/diagnostics/model_report.pdf


In [1]:
from ultralytics import YOLO
import torch
import os
import glob

# --- Config ---
map_folder = "Bear_Lane"
image_dir = rf"C:\QGIS\{map_folder}\tiled\images"

# Absolute path to trained model
model_path = r"C:\QGIS\runs\segment\train\main_model\weights\best.pt"

# Output folder for predictions
output_dir = rf"C:\QGIS\{map_folder}\predictions"
output_name = "predict_txt"

# --- Load model directly ---
model = YOLO(model_path)

# Output folder for predictions
output_dir = rf"C:\QGIS\{map_folder}\predictions"
output_name = "predict_txt"

# --- Optional: Clean up old predictions ---
label_dir = os.path.join(output_dir, output_name, "labels")
if os.path.exists(label_dir):
    old_labels = glob.glob(os.path.join(label_dir, "*.txt"))
    for f in old_labels:
        os.remove(f)
    print(f"🧹 Cleared {len(old_labels)} old prediction files from: {label_dir}")

# --- Predict with streaming ---
results = model.predict(
    source=image_dir,
    imgsz=640,
    conf=0.1,
    save=False,
    save_txt=True,
    save_conf=False,
    retina_masks=True,
    exist_ok=True,
    project=output_dir,
    name=output_name,
    device="cuda" if torch.cuda.is_available() else "cpu",
    stream=True
)

# Force the generator to run
for _ in results:
    pass

print(f"✅ Inference complete. Labels saved to: {output_dir}\\{output_name}\\labels\\")

🧹 Cleared 270 old prediction files from: C:\QGIS\Bear_Lane\predictions\predict_txt\labels

image 1/1470 C:\QGIS\Bear_Lane\tiled\images\chip_0.png: 640x640 (no detections), 65.5ms
image 2/1470 C:\QGIS\Bear_Lane\tiled\images\chip_1.png: 640x640 (no detections), 62.1ms
image 3/1470 C:\QGIS\Bear_Lane\tiled\images\chip_10.png: 640x640 (no detections), 73.4ms
image 4/1470 C:\QGIS\Bear_Lane\tiled\images\chip_100.png: 640x640 (no detections), 72.1ms
image 5/1470 C:\QGIS\Bear_Lane\tiled\images\chip_1000.png: 640x640 7 PVegs, 66.3ms
image 6/1470 C:\QGIS\Bear_Lane\tiled\images\chip_1001.png: 640x640 10 PVegs, 78.3ms
image 7/1470 C:\QGIS\Bear_Lane\tiled\images\chip_1002.png: 640x640 18 PVegs, 84.3ms
image 8/1470 C:\QGIS\Bear_Lane\tiled\images\chip_1003.png: 640x640 6 PVegs, 1 Water, 77.7ms
image 9/1470 C:\QGIS\Bear_Lane\tiled\images\chip_1004.png: 640x640 8 PVegs, 72.9ms
image 10/1470 C:\QGIS\Bear_Lane\tiled\images\chip_1005.png: 640x640 12 PVegs, 72.7ms
image 11/1470 C:\QGIS\Bear_Lane\tiled\image

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio
from tqdm import tqdm

# --- Config ---
map_folder = "Bear_Lane"
base_dir = f"C:/QGIS/{map_folder}"
mask_dir = os.path.join(base_dir, "predictions", "predict_txt", "labels")
tile_metadata_path = os.path.join(base_dir, "tiled", "tile_metadata.csv")
raster_shape_path = os.path.join(base_dir, "tiled", "raster_shape.txt")
tif_path = os.path.join(base_dir, f"{map_folder}.tiff")
shapefile_path = os.path.join(base_dir, f"{map_folder} Segmentation.shp")

chip_size = 1024        # Original chip size used during tiling
inference_size = 640    # YOLOv8 inference resolution

# Optional: Remap YOLO class IDs → Original class IDs
reverse_remap = {
    0: 1,  # Road
    1: 2,  # PVeg
    2: 3   # Water
}

# --- Load metadata and raster georeferencing ---
tile_meta = pd.read_csv(tile_metadata_path)
with open(raster_shape_path, "r") as f:
    height, width = map(int, f.read().strip().split(","))

with rasterio.open(tif_path) as src:
    transform = src.transform
    crs = src.crs

# --- Collect all polygons from predicted labels ---
features = []

for _, row in tqdm(tile_meta.iterrows(), total=len(tile_meta), desc="Stitching Tiles"):
    fname, x, y = row["filename"], int(row["x"]), int(row["y"])
    label_path = os.path.join(mask_dir, fname.replace(".png", ".txt"))
    if not os.path.exists(label_path):
        continue

    with open(label_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 7:
            continue  # Skip degenerate polygons

        cls_id = int(float(parts[0]))  # Already YOLO-aligned: 0 = Road, 1 = PVeg, 2 = Water

        coords = list(map(float, parts[1:]))
        pts = np.array(coords, dtype=np.float32).reshape(-1, 2)
        pts *= chip_size  # Rescale to original resolution

        pts[:, 0] += x
        pts[:, 1] += y

        geo_pts = [rasterio.transform.xy(transform, y_, x_, offset='center') for x_, y_ in pts]
        poly = Polygon(geo_pts)

        if poly.is_valid and poly.area > 0:
            features.append({
                "geometry": poly,
                "class_id": reverse_remap.get(cls_id, cls_id)  # Optional remap
            })

# --- Export to shapefile ---
gdf = gpd.GeoDataFrame(features, crs=crs)
gdf.to_file(shapefile_path)

print(f"✅ Shapefile saved to: {shapefile_path}")

Stitching Tiles: 100%|█████████████████████████████████████████████████████████████| 1470/1470 [18:28<00:00,  1.33it/s]


✅ Shapefile saved to: C:/QGIS/Bear_Lane\Bear_Lane Segmentation.shp
